**This kernel mainly covers:**<br>
<font color='#0073e6'>
    <ol>
        <b><li>Data Cleaning</li></b>
        <b><li>Data Munging</li></b>
        <b><li>EDA</li></b>
    </ol>
</font>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools as itr
import datetime as datetime

In [ ]:
data_file_path = '/kaggle/input/top50spotify2019/'

In [ ]:
# Please note that we are using python engine to parse the file data instead of c engine.
# read_csv() throws error when we use c-engine.

top50_data = pd.read_csv(data_file_path + 'top50.csv', engine = 'python')
top50_data.shape

In [ ]:
top50_data.info()

In [ ]:
top50_data.head()

In [ ]:
# Rename column.
top50_data.rename(columns = {'Unnamed: 0':'id'}, inplace = True)

# Change name of columns to lower case.
top50_data.columns = top50_data.columns.str.replace('.', '').str.lower()

top50_data.columns

In [ ]:
top50_data.head()

In [ ]:
def get_df_summary(df):
    
    '''
    
    This function is used to summarise especially unique value count, NaN values count, 
    Blank values count and data type for variable
    
    '''
    
    unq_val_cnt_df = pd.DataFrame(df.nunique(), columns = ['unq_val_cnt'])
    unq_val_cnt_df.reset_index(inplace = True)
    unq_val_cnt_df.rename(columns = {'index':'variable'}, inplace = True)
    unq_val_cnt_df = unq_val_cnt_df.merge(df.dtypes.reset_index().rename(columns = {'index':'variable', 0:'dtype'}),
                                          on = 'variable')
    unq_val_cnt_df = unq_val_cnt_df.sort_values(by = 'unq_val_cnt', ascending = False)
    
    nans_df = pd.DataFrame(df.isna().sum(), columns = ['no_nans']).reset_index()
    nans_df.rename(columns = {'index':'variable'}, inplace = True)
    
    blank_df = pd.DataFrame((df.select_dtypes(include = 'O') == '').sum(), columns = ['no_blanks']).reset_index()
    blank_df.rename(columns = {'index':'variable'}, inplace = True)
    
    neg_df = pd.DataFrame((df.select_dtypes(include = ['int64', 'float64']).lt(0)).sum(), columns = ['no_neg']).reset_index()
    neg_df.rename(columns = {'index':'variable'}, inplace = True)
    
    summ_df = unq_val_cnt_df.merge(nans_df, how = 'outer', on = 'variable')
    summ_df = summ_df.merge(blank_df, how = 'outer', on = 'variable')
    summ_df = summ_df.merge(neg_df, how = 'outer', on = 'variable')
    
    summ_df.fillna(-1, inplace = True)
    
    return summ_df

In [ ]:
top50_summ = get_df_summary(top50_data)
top50_summ

In [ ]:
print('No. of variables with NaN : {}'.format(sum(top50_summ['no_nans'] > 0)))
print('No. of variables with Blanks : {}'.format(sum(top50_summ['no_blanks'] > 0)))
print('No. of variables with Neg. values : {}'.format(sum(top50_summ['no_neg'] > 0)))

**Observation:**

Based on the data summary (above), we can conclude that we do not have either missing values or blank values to handle; neither we see negative values expectedly in any of the given variables.

In [ ]:
# List the variable(s) containing negative values.

top50_summ.loc[top50_summ['no_neg'] > 0, 'variable']

**Variable : loudnessdb**

In [ ]:
top50_data['loudnessdb'].describe()

Of course, there is one variable **loudnessdb** with negative values; here db stands for **Decibel**. For the benefit of this data analysis, let us look at what actually a **decibel** is and try to understand the significance of decibles measured in positive and negative values. <br>

#### 1. What is Decibel?<br><br>

<center><i>"Decibel is a unit used to measure the intensity of a sound or the power level of an electrical signal by comparing it with a given level on a logarithmic scale."</i></center>
<p style='text-align: right;'>- According to Oxford Dictionary</p>

&nbsp;&nbsp;&nbsp;Basically, **its the degree of loudness**. **One decibel** is one tenth (deci-) of **one bel**, named in honor of **Alexander Graham Bell**. Symbolically, it is represented as **dB**.<br> 

#### 2. What are the range of Decibel values? <br>

&nbsp;&nbsp;&nbsp;Decibel can be used to express either of the following:<br> 
- Change in value: Represents a change in value with reference to 0 (Zero) which means it can attain a positive or a negative value.
- An absolute value: Represents the ratio of a value to a fixed reference value.<br><br>

&nbsp;&nbsp;&nbsp;For our data analysis, we need to understand what does a Zero, Positive and a Negative dB value indicate. <br>
- Zero dB : **Threshold** of human hearing i.e. the softest sound the human ear can hear (without any artificial help).
- Positive dB : indicate sound is **few times louder** than the threshold.
- Negative dB : indicate sound is **few times softer** than the threshold.

In [ ]:
print('No. of human audible songs in the given data set: {}'.format(top50_summ.at[(top50_summ['variable'] == 'loudnessdb').idxmax(), 'no_neg']))

All values of **loudnessdb** are negative, which means that all 50 songs are human audible.

In [ ]:
# Check duplicate records.

print('No. of duplicate obs. : {}'.format(top50_data.duplicated().sum()))

In [ ]:
top50_data.describe()

**Variable : beatsperminute**

**Info-bit:**

**Beats per minute (bpm)** a.k.a as **Tempo** represents speed of a music piece. Most commonly, Tempo is expressed in ***Beats per seconds (bps)*** or ***Hertz (Hz)***. <br><br>
<center><b>$1 bpm = \frac{1}{60} Hz$</b></center><br>
Decision about where to convert bpm to Hz will be taken up in the later part of this notebook if required.

In [ ]:
top50_data['beatsperminute'].describe()

**Assumptions:** <br>
1. 1-100 is the range in which the values captured for the variables **energy**, **danceability**, **liveness**, **valence**, **acousticness**, **speechiness** and **popularity**.
2. **length** values are in seconds.

**Variable : len_min_secs**

In [ ]:
# Convert seconds value of length variable into minutes and seconds.

top50_data['len_min_secs'] = top50_data['length'].apply(lambda x : datetime.time(divmod(x, 60)[0], divmod(x, 60)[1]))

top50_data.head()

**1. Who are the Top-3 artists with maximum number of songs in the list?**

In [ ]:
tmp_df = pd.DataFrame(top50_data.groupby('artistname')['id'].count())
tmp_df.reset_index(inplace = True)
tmp_df.rename(columns = {'id':'count'}, inplace = True)
tmp_df = tmp_df.sort_values(by = 'count', ascending = False).head(3)
tmp_df

In [ ]:
fig = px.scatter(data_frame = tmp_df,
                 x = 'artistname',
                 y = 'count',
                 color = 'artistname',
                 labels = {'artistname':'Artist Name', 'count':'No. of songs'},
                 size = 'count')
fig.update_xaxes(title_text = 'Artist Name')
fig.update_yaxes(title_text = 'No. of songs', dtick = 1)
fig.update_layout(title_text = 'Top-3 artists with maximum of songs')
fig.show()

**Ed Sheeran** tops the maximum number of songs in the list.

**2. Which Top-3 genres having maximum number of songs in this list?**

In [ ]:
tmp_df = pd.DataFrame(top50_data.groupby('genre')['id'].count())
tmp_df.reset_index(inplace = True)
tmp_df.rename(columns = {'id':'count'}, inplace = True)
tmp_df = tmp_df.sort_values(by = 'count', ascending = False).head(3)

tmp_df

In [ ]:
fig = px.scatter(data_frame = tmp_df,
                 x = 'genre',
                 y = 'count',
                 color = 'genre',
                 labels = {'genre':'Genre', 'count':'No. of songs'},
                 size = 'count'
                 )
fig.update_xaxes(title_text = 'Genre')
fig.update_yaxes(title_text = 'No. of songs', dtick = 1)
fig.update_layout(title_text = 'Top-3 genres with maximum of songs')
fig.show()

**3. Which are the Top-3 lengthiest songs in this list?**

In [ ]:
tmp_df = top50_data.sort_values(by = 'length', ascending = False)[['trackname', 'artistname', 'genre', 'len_min_secs', 'length']]
tmp_df = tmp_df.head(3)

tmp_df

In [ ]:
fig = px.bar(data_frame = tmp_df,
             x = 'length',
             y = 'trackname',
             color = 'genre',
             labels = {'genre':'Genre', 'length':'Duration', 'trackname':'Track Name'},
             orientation = 'h',
             text = 'len_min_secs'
            )
fig.update_xaxes(title_text = 'Duration (in seconds)')
fig.update_yaxes(title_text = 'Track Name')
fig.update_layout(title_text = 'Top-3 lengthiest songs',
                  yaxis = {'categoryorder':'total ascending'})

fig.show()

**4. Which are the Top-10 most popular songs in this list?**

In [ ]:
tmp_df = top50_data.sort_values(by = 'popularity', ascending = False)[['trackname', 'artistname', 'genre', 'len_min_secs', 'popularity', 'length']]
tmp_df = tmp_df.head(10)

tmp_df

In [ ]:
fig = px.scatter(data_frame = tmp_df,
                 x = 'genre',
                 y = 'popularity',
                 labels = {'genre':'Genre', 'popularity':'Popularity', 'length':'Duration'},
                 size = 'length'
                 )
fig.update_xaxes(title_text = 'Genre')
fig.update_yaxes(title_text = 'Popularity', dtick = 1)
fig.update_layout(title_text = 'Top-10 most popular songs')
fig.show()

**Observations:**<br>
Duration of a song does not decide the popularity of a song. There are songs which are more popular and at the same time shorter in duration compared to other songs which are longer in duration.

In [ ]:
cut_labels = [str(i) + '-' + str(i + 10) for i in list(range(0, 100, 10))]
cut_bins = list(range(0, 110, 10))

**Variable : energy_bin**

In [ ]:
top50_data['energy_bin'] = pd.cut(top50_data['energy'], bins = cut_bins, labels = cut_labels)

**Variable : danceability_bin**

In [ ]:
top50_data['danceability_bin'] = pd.cut(top50_data['danceability'], bins = cut_bins, labels = cut_labels)

**Variable : liveness_bin**

In [ ]:
top50_data['liveness_bin'] = pd.cut(top50_data['liveness'], bins = cut_bins, labels = cut_labels)

**Variable : valence_bin**

In [ ]:
top50_data['valence_bin'] = pd.cut(top50_data['valence'], bins = cut_bins, labels = cut_labels)

**Variable : acousticness_bin**

In [ ]:
top50_data['acousticness_bin'] = pd.cut(top50_data['acousticness'], bins = cut_bins, labels = cut_labels)

**Variable : speechiness_bin**

In [ ]:
top50_data['speechiness_bin'] = pd.cut(top50_data['speechiness'], bins = cut_bins, labels = cut_labels)

**Variable : length_bin**

In [ ]:
cut_labels = [str(i) + '-' + str(i + 30) for i in list(range(0, 330, 30))]
top50_data['length_bin'] = pd.cut(top50_data['length'], bins = list(range(0, 360, 30)), labels = cut_labels)

**5. What is more important for listeners of the songs?**

In [ ]:
song_pop_prop_df = pd.DataFrame()
cols_to_plot = ['energy', 'danceability', 'liveness', 'valence', 'acousticness', 'speechiness', 'popularity']

for col in cols_to_plot:
    tmp_df = pd.DataFrame(top50_data.loc[:, col]).reset_index(drop = True)
    tmp_df.rename(columns = {col:'property_val'}, inplace = True)
    tmp_df['property'] = col
    song_pop_prop_df = pd.concat([song_pop_prop_df, tmp_df], sort = False)

song_pop_prop_df.fillna(value = {'property_val':0}, inplace = True)

fig = px.box(data_frame = song_pop_prop_df,
             x = 'property',
             y = 'property_val',
             labels = {'property_val':'Property Value', 'property':'Property'})
fig.update_layout(title_text = 'Property Composition of Top-50 songs')
fig.show()

**Observations:**<br><br>
*Note : Range of values (Spread of data) for each of the property gives us an indication about significance of each of these properties that makes listener to listen to these songs more number of times which inturn makes these songs to enter the Top-50 list.* <br><br>

1. **Popularity** of the Top-50 songs ranges between **[70-95]**.
2. Songs with more **Energy, Danceability, Valence & Acousticness** have made it to the Top-50 list.
3. More number of listeners **liked "Studio recorded" songs than "Live recorded"**.
4. Songs with **less lyrics** compared to music are **more** likened.
5. **Valence, Energy & Danceability** among other song properties are being likened by most of the listeners.

*Note : **Valence** is a measure of musical positiveness measured between 0.0 and 1.0. Song with high valence sound more positive while song with low valence sound more negative.*

**6. How song popularity varies with variation in song properties ?**

In [ ]:
fig = make_subplots(rows = 4, 
                    cols = 2, 
                    shared_yaxes = True, 
                    vertical_spacing = .095, 
                    y_title = 'Popularity', 
                    row_heights = [.8, .8, .8, .8])

rol_col_ids = list(itr.product(range(1, 5), range(1, 3)))
cols_to_plot = ['energy_bin', 'danceability_bin', 'liveness_bin', 'valence_bin', 'acousticness_bin', 
                'speechiness_bin', 'length_bin']

for idx in range(len(cols_to_plot)):
    
    col = cols_to_plot[idx]
    
    tmp_df = pd.DataFrame(top50_data.groupby(col)['popularity'].median().round())
    tmp_df = tmp_df.reset_index()
    
    label_name = col.split('_')[0].title()
    
    fig.append_trace(go.Scatter(x = tmp_df[col], y = tmp_df['popularity'], name = label_name), 
                     row = rol_col_ids[idx][0], 
                     col = rol_col_ids[idx][1]
                     )
    
    fig.update_layout(title_text = 'Song Popularity v/s Song Properties', height = 900, width = 900)

fig.show()

**Observation:**<br><br>
Energy/Danceability/Liveness/Valence/Acousticness/Speechiness/Length : It was expected that<br>
1. None of these song properties are found to be individually influencing popularity of the songs.
2. Combination of two or more of these properties makes listeners to a song more frequently.

**Variable : popularity_bin**

In [ ]:
cut_labels = [str(i) + '-' + str(i + 10) for i in list(range(0, 100, 10))]
top50_data['popularity_bin'] = pd.cut(top50_data['popularity'], bins = list(range(0, 110, 10)), labels = cut_labels)

**7. Do we have a right mixture of song properties that make(s) a song or songs more popular?**

In [ ]:
song_pop_prop_df = pd.DataFrame()
cols_to_plot = ['energy', 'danceability', 'liveness', 'valence', 'length', 'acousticness', 'speechiness']

for col in cols_to_plot:
    tmp_df = pd.DataFrame(top50_data.groupby('popularity_bin')[col].mean().round()).reset_index()
    tmp_df.rename(columns = {col:'property_val'}, inplace = True)
    tmp_df['property'] = col
    song_pop_prop_df = pd.concat([song_pop_prop_df, tmp_df], sort = False)

song_pop_prop_df.fillna(value = {'property_val':0}, inplace = True)

fig = px.bar(data_frame = song_pop_prop_df,
             x = 'popularity_bin',
             y = 'property_val',
             color = 'property',
             barmode = 'relative',
             labels = {'popularity_bin':'Popularity', 'property':'Property', 'property_val':'Property Value'}
             )

fig.update_xaxes(title_text = 'Popularity')
fig.update_yaxes(title_text = 'Property Value')
fig.update_layout(title_text = 'Contribution of Song Properties in popularizing songs')
fig.show()

**Observations:**<br><br>

As we do not have list of songs that did not make to Top-50 list, we cannot do a comparitive study between Top-50 songs and the one which are not in this list.<br>

However, we can compare the least (**[60-70]**) & most (**[90-100]**) popular songs of this list.

**8. Analysis and Comparison of Top-10 and Bottom-10 songs.**

In [ ]:
pop_prop_df = pd.DataFrame()

for pop_bin in ['60-70', '90-100']:
    tmp_df = top50_data.loc[top50_data['popularity_bin'] == pop_bin, cols_to_plot].median().round().reset_index()
    tmp_df['pop_bin'] = pop_bin
    pop_prop_df = pd.concat([pop_prop_df, tmp_df])
    
pop_prop_df.rename(columns = {0:'prop_value', 'index':'prop'}, inplace = True)

fig = px.bar(pop_prop_df, x = 'prop', y = 'prop_value', color = 'pop_bin', barmode = 'group')

fig.update_xaxes(title_text = 'Song Property of Top-10 & Bottom-10 songs')
fig.update_yaxes(title_text = 'Song Property Value')
fig.update_layout(title_text = 'Song Properties for Lowest & Highest Popularity Songs',
                  xaxis = {'categoryorder':'total descending'})

fig.show()

**Observations:**<br><br>

In this plot, we see a quite number of surprising elements which I thought would not be the reason(s) for the songs to be in the 90-100 popularity list. Let's derive those points from the plot.<br><br>

1. Songs in 90-100 popularity list were expected to have **less speechiness, liveness & length**. But, its the other way around.
2. Songs in 60-70 popularity list were expected to have **less energy, valence & acousticness**. But, its the other way around.
3. **High energy** *and/or* **valence** *and/or* **danceability** *and/or* **acousticness** were the properties most anticipated from the songs appearning in 90-100 popularity range.
4. And the songs with **more speechiness, liveness** *&* **length** were expected to less popular compared to the songs of 90-100 popularity range.

Let's see if either beatsperminute or loudnessdb or both could be enticing the listeners to listen to these top-50 songs or 90-100 songs more.

**9. Analysis of properties of 10-5-bunched Top-50 songs**

Steps followed to perform this analysis:<br><br>
1. Order given Top-50 songs data descending by popularity of the song.
2. Group the 50 songs into 5 groups with each containing 10 songs in the same order.
3. Compute median for each of these 5 groups individually for the selected properties.
4. Then, compare variations in danceability, liveness, valence, length, acousticness, speechiness, beatsperminute, loudnessdb & energy between each of the 5 groups.

In [ ]:
tmp_df = top50_data.sort_values(by = 'popularity', ascending = False)[['danceability', 'liveness', 'valence', 'length',
                                                                       'acousticness', 'speechiness', 'popularity',
                                                                       'beatsperminute', 'loudnessdb', 'energy']]
tmp_df.reset_index(drop = True, inplace = True)
tmp_df.reset_index(inplace = True)

cut_labels = [str(i + 1) + '-' + str(i + 10) for i in list(range(0, 50, 10))]
cut_bins = list(range(-1, 51, 10))

tmp_df['bunch'] = pd.cut(tmp_df['index'], bins = cut_bins, labels = cut_labels)

In [ ]:
# tmp_df.head()

In [ ]:
songs_bunch_df = tmp_df.loc[:, ~tmp_df.columns.isin(['index'])].groupby('bunch').median().round().reset_index()
songs_bunch_df.reset_index(inplace = True, drop = True)

In [ ]:
# songs_bunch_df.head(20)

In [ ]:
trf_songs_bunch_df = pd.DataFrame()
curr_songs_bunch_df = pd.DataFrame()

for col in songs_bunch_df.columns[1:]:
    curr_songs_bunch_df['bunch'] = songs_bunch_df['bunch']
    curr_songs_bunch_df['property'] = col
    curr_songs_bunch_df['property_value'] = songs_bunch_df[col]
    
    trf_songs_bunch_df = trf_songs_bunch_df.append(curr_songs_bunch_df, ignore_index = True)

In [ ]:
# trf_songs_bunch_df.head()

In [ ]:
fig = make_subplots(rows = 2, 
                    cols = 3, 
                    shared_yaxes = True, 
                    vertical_spacing = .1, 
                    y_title = 'Property Value', 
                    row_heights = [.8, .8], 
                    shared_xaxes = True)

rol_col_ids = list(itr.product(range(1, 3), range(1, 4)))
bunch_to_plot = trf_songs_bunch_df['bunch'].unique().tolist()
cols_to_plot = ['energy', 'danceability', 'liveness', 'valence', 'acousticness', 'speechiness']

for idx in range(len(bunch_to_plot)):
    
    bunch = bunch_to_plot[idx]
    
    tmp_df = trf_songs_bunch_df.loc[(trf_songs_bunch_df['bunch'] == bunch) & (trf_songs_bunch_df['property'].isin(cols_to_plot)), ['property', 'property_value']]
    
    fig.append_trace(go.Scatter(x = tmp_df['property'], y = tmp_df['property_value'], name = bunch), 
                     row = rol_col_ids[idx][0], 
                     col = rol_col_ids[idx][1]
                     )

fig.update_layout(title_text = 'Song Properties of 5 bunch of songs', width = 900, height = 600)

fig.show()

**Observations:**<br><br>

1. <u>Danceability:</u><br> 
   a. is **neither too high nor too low** for the **Top-10 songs**.<br>
   b. is highest for the **Bottom-10 songs**.<br>
2. <u>Liveness:</u><br>
   a. is **neither too high nor too low** for the **Top-10 songs**.<br>
   b. is **lowest** for the **Bottom-10 songs**.<br>
   c. is **highest** for the **3rd Top-10 songs** list.<br>
3. <u>Valence:</u><br>
   a. **Highest** for the **Bottom-10 songs**.<br>
   b. **Lowest** for the **Top-10 songs**.<br>
4. <u>Acousticness:</u><br>
   a. **Highest** for the **3rd Top-10 songs**.<br>
   b. **Lowest** for the **Top-10 songs**.<br>
5. <u>Speechiness:</u><br>
   a. **Highest** for the **Top-10 songs**.
6. <u>Energy:</u><br>
   a. **Lowest** for the **Top-10 songs**.<br>
   b. **Highest** for the **2nd Top-10 songs**.<br>

In [ ]:
fig = make_subplots(rows = 2, 
                    cols = 3, 
                    shared_yaxes = True, 
                    vertical_spacing = .1, 
                    y_title = 'Popularity', 
                    row_heights = [.8, .8],
                   shared_xaxes = True)

rol_col_ids = list(itr.product(range(1, 3), range(1, 4)))
bunch_to_plot = trf_songs_bunch_df['bunch'].unique().tolist()
cols_to_plot = ['length', 'beatsperminute', 'loudnessdb']

for idx in range(len(bunch_to_plot)):
    
    bunch = bunch_to_plot[idx]
    
    tmp_df = trf_songs_bunch_df.loc[(trf_songs_bunch_df['bunch'] == bunch) & (trf_songs_bunch_df['property'].isin(cols_to_plot)), ['property', 'property_value']]
    
    fig.append_trace(go.Scatter(x = tmp_df['property'], y = tmp_df['property_value'], name = bunch), 
                     row = rol_col_ids[idx][0], 
                     col = rol_col_ids[idx][1]
                     )

fig.update_layout(title_text = 'Song Properties of bunch of songs', width = 900, height = 600)

fig.show()

**Observations:**<br><br>

1. <u>Length</u>:<br>
   a. **3rd Top-10 songs** are the **lengthiest** among all other songs.<br>
   b. **Top-10 songs** are the **shortest** ones.
2. <u>Beats per Minute</u>:<br>
   a. **Top-10 songs** have the **highest** beats per minute.<br>
   b. **2nd Top-10 songs** have the **least** beats per minute.<br>
3. <u>Loudness in DB</u>:<br>
   a. is either -5 or -6 for the all Top-50 songs.

**Final Conclusion:**<br><br>

1. Songs which made to the top-50 or top-10 list not merely based on:<br>
   a. increase or decrease of proportion of certain factor(s) in the songs.<br>
   b. combination of the factors in the given data set. There may be some more useful features (other than one given) which<br> we can gather and analyse.<br>
2. None of the factor(s) are found to be either increasing or decreasing gradually from top-10 to bottom-10 list. Certain<br> factors in between these different top-10s have been either lowest or highest.<br>
3. Not all the song properties were neither higher nor lower for top-10 songs. Same applies for the bottom-10 songs.<br>
4. In spite of all this, there are few factors which stand-out for top-10 and bottom-10 songs. They are:<br>
   a. For Top-10 songs:<br>
      &emsp;(i) Danceability and Liveness is average.<br>
      &emsp;(ii) Valence, Acousticness and Energy is lowest.<br>
      &emsp;(iii) Length of the songs is shortest.<br>
      &emsp;(iv) Speechiness and beats per minute are highest.<br>
   b. For Bottom-10 songs:<br>
      &emsp;(i) Danceability & Valence are highest.<br>
      &emsp;(ii) Valence is lowest.<br><br>
      
**"No data analysis concludes unless we conclude it."**<br><br> 
Therefore, I conclude this data analysis by saying that there exists no magical recipe with a perfect blend of the song properties (in the given data set) that gives us a top-10 or top-50 song. Factors (which I feel are important are) such as:<br> 
- Most trending songs at the time of release of an album.
- Most trending type of music at the time of release of an album.
- Advances in the technology affects music making and music industry the most.
- Marketting of the songs through promotional videos and making of video songs.

Your upvote encourages and motivates me to keep going. And, feedback gives me an opportunity to look with-in myself and to overcome mistakes and lackness if any. Therefore, I request you to upvote this kernel if you like my work and leave a feedback.<br><br>

Thank You.